## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the Weather_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Jamestown,US,2022-25-09/02/22 21:25:31,42.0970,-79.2353,82.02,39,0,10.36,clear sky
1,1,San Francisco,US,2022-27-09/02/22 21:27:18,37.7749,-122.4194,96.03,74,20,28.77,few clouds
2,2,Codrington,AU,2022-27-09/02/22 21:27:23,-38.2667,141.9667,48.06,87,77,8.75,broken clouds
3,3,Salalah,OM,2022-27-09/02/22 21:27:24,17.0151,54.0924,78.89,83,75,5.75,broken clouds
4,4,Kaniama,CD,2022-27-09/02/22 21:27:24,-7.5667,24.1833,68.59,87,100,6.76,light rain


In [3]:
# Ask the customer to specify the temperature range for their trip

min_temp = float(input('What is the minimum temperature you would like for your trip?'))

max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?65.8
What is the maximum temperature you would like for your trip?89.4


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

In [5]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                593
City                   593
Country                590
Date                   593
Lat                    593
Lng                    593
Max Temp               593
Humidity               593
Cloudiness             593
Wind Speed             593
Weather Description    593
dtype: int64

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

clean_df.count()

City_ID                590
City                   590
Country                590
Date                   590
Lat                    590
Lng                    590
Max Temp               590
Humidity               590
Cloudiness             590
Wind Speed             590
Weather Description    590
dtype: int64

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Jamestown,US,82.02,clear sky,42.0970,-79.2353,
3,Salalah,OM,78.89,broken clouds,17.0151,54.0924,
4,Kaniama,CD,68.59,light rain,-7.5667,24.1833,
5,Butaritari,KI,81.95,clear sky,3.0707,172.7902,
7,Rikitea,PF,71.04,broken clouds,-23.1203,-134.9692,
9,Yambio,SS,67.28,overcast clouds,4.5721,28.3955,
11,Yatou,CN,69.39,overcast clouds,37.1500,122.3833,
14,Ribeira Grande,PT,72.52,overcast clouds,38.5167,-28.7000,
16,Vaini,TO,73.56,few clouds,-21.2000,-175.2000,
17,Mahon,ES,79.36,few clouds,39.8885,4.2658,


In [10]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Add a progress counter
ctr = 0

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # get the location lat & lng
    lat = row['Lat']
    lng = row['Lng']
    
    city = row['City']
    country = row['Country']
    
    # Add lat and lng to a location key for our search params
    params['location'] = f'{lat},{lng}'
    
    # Make the request and get JSON response data
    hotels = requests.get(base_url, params=params).json()
    time.sleep(1)
    
    # print the progress counter in increments of 10
    if (ctr % 10 ==0):
        print(f'Counter = {ctr}')
        
    try:
        # Identify the hotels from the results and store the names
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print(f'Location: {city}, {country} - Hotel not found....  Skipping.')
    
    ctr += 1

Counter = 0
Location: Kaniama, CD - Hotel not found....  Skipping.
Counter = 10
Counter = 20
Location: Puerto Lleras, CO - Hotel not found....  Skipping.
Counter = 30
Location: Poya, NC - Hotel not found....  Skipping.
Counter = 40
Counter = 50
Location: Fonte Boa, BR - Hotel not found....  Skipping.
Counter = 60
Counter = 70
Counter = 80
Location: Gao, ML - Hotel not found....  Skipping.
Location: Avera, US - Hotel not found....  Skipping.
Location: Panguna, PG - Hotel not found....  Skipping.
Location: Raga, SS - Hotel not found....  Skipping.
Counter = 90
Location: Micheweni, TZ - Hotel not found....  Skipping.
Counter = 100
Counter = 110
Counter = 120
Location: Albany, US - Hotel not found....  Skipping.
Location: Witu, KE - Hotel not found....  Skipping.
Counter = 130
Location: Gubskaya, RU - Hotel not found....  Skipping.
Counter = 140
Counter = 150
Location: Bonfim, BR - Hotel not found....  Skipping.
Counter = 160
Location: Goundam, ML - Hotel not found....  Skipping.
Counter =

In [20]:
# Drop the rows where there is no Hotel Name
hotel_df.drop(hotel_df[ hotel_df['Hotel Name'] == ""].index, inplace=True)

hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Jamestown,US,82.02,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
3,Salalah,OM,78.89,broken clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
5,Butaritari,KI,81.95,clear sky,3.0707,172.7902,Isles Sunset Lodge
7,Rikitea,PF,71.04,broken clouds,-23.1203,-134.9692,People ThankYou
9,Yambio,SS,67.28,overcast clouds,4.5721,28.3955,Naivasha Hotel
...,...,...,...,...,...,...,...
1090,Dunkwa,GH,71.71,broken clouds,5.9656,-1.7800,Tim Freeman Gold City Hotel
1092,Oistins,BB,86.09,few clouds,13.0667,-59.5333,Butterfly Beach Hotel
1095,Banes,CU,82.81,overcast clouds,20.9625,-75.7186,Hostal Iván y yailet
1097,Katobu,ID,73.85,few clouds,-4.9408,122.5276,La Ode Husein


In [21]:
# Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Assign the figure variable for gmaps - Configure the figure for fit and zoom
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the dat
fig

Figure(layout=FigureLayout(height='420px'))